In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
from scipy import stats
from scipy.spatial import distance
import random
random.seed(8888)

In [2]:
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data0)
y_pred = clf.predict(data0)
data0['ISO'] = y_pred
anomalies = data0[data0.ISO == -1]

In [8]:
anomalies = anomalies[["bytes","pkts","dur","rate"]]

In [6]:
# 3. run KSE 
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [] # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        #kse_score[i] = average
        kse_score.append((i, average))
        if (i%1000 == 0):
            print('loop %s' % i)
    return kse_score 

In [9]:
kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))

half way there
loop 0


In [11]:
# 4. get result
index_normal = [] 
for i in kse_result:
    if i[1] < 0.5:
        index_normal.append(i[0])
# final result is a data set of the anomalies of the anomalies
monday = anomalies.iloc[index_normal]

In [12]:
data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data0)
y_pred = clf.predict(data0)
data0['ISO'] = y_pred
anomalies = data0[data0.ISO == -1]
anomalies = anomalies[["bytes","pkts","dur","rate"]]
kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
# 4. get result
index_normal = [] 
for i in kse_result:
    if i[1] < 0.5:
        index_normal.append(i[0])
# final result is a data set of the anomalies of the anomalies
tuesday = anomalies.iloc[index_normal]

half way there
loop 0


In [14]:
len(tuesday)

339

In [17]:
stats.ks_2samp(monday.rate,tuesday.rate)

Ks_2sampResult(statistic=0.38074787437098734, pvalue=7.8216954390668168e-17)

In [19]:
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data0)
y_pred = clf.predict(data0)
data0['ISO'] = y_pred
anomalies = data0[data0.ISO == -1]
anomalies = anomalies[["bytes","pkts","dur","rate"]]
kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
# 4. get result
index_normal = [] 
for i in kse_result:
    if i[1] > 0.5:
        index_normal.append(i[0])
# final result is a data set of the anomalies of the anomalies
monday_anom = anomalies.iloc[index_normal]

half way there
loop 0


In [20]:
data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data0)
y_pred = clf.predict(data0)
data0['ISO'] = y_pred
anomalies = data0[data0.ISO == -1]
anomalies = anomalies[["bytes","pkts","dur","rate"]]
kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
# 4. get result
index_normal = [] 
for i in kse_result:
    if i[1] > 0.5:
        index_normal.append(i[0])
# final result is a data set of the anomalies of the anomalies
tuesday_anom = anomalies.iloc[index_normal]

half way there
loop 0


In [26]:
stats.ks_2samp(monday_anom.rate,tuesday_anom.rate)

Ks_2sampResult(statistic=0.88235294117647056, pvalue=0.0015833340948373303)

In [106]:

def ks_stat_anom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
    clf.fit(data0)
    y_pred = clf.predict(data0)
    data0['ISO'] = y_pred
    anomalies = data0[data0.ISO == -1]
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]

    # 4. get result
    index_normal = [] 
    for i in kse_result:
        if i[1] < 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies less the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom

def ks_stat_anomanom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
    clf.fit(data0)
    y_pred = clf.predict(data0)
    data0['ISO'] = y_pred
    anomalies = data0[data0.ISO == -1]
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]
    index_normal = [] 
    for i in kse_result:
        if i[1] > 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom
    

In [150]:
##### LOF
def ks_stat_anom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    y_pred = clf.fit_predict(data0)
    data0['LOF'] = y_pred
    anomalies = data0[data0.LOF == -1]
    
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]

    # 4. get result
    index_normal = [] 
    for i in kse_result:
        if i[1] < 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies less the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom

def ks_stat_anomanom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    y_pred = clf.fit_predict(data0)
    data0['LOF'] = y_pred
    anomalies = data0[data0.LOF == -1]
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]
    index_normal = [] 
    for i in kse_result:
        if i[1] > 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom
    

In [204]:
# ece
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
a = ks_stat_anom(data0)
aa = ks_stat_anomanom(data0)
data1 = pd.read_csv('~/Desktop/data/data02_14_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
b = ks_stat_anom(data1)
bb = ks_stat_anomanom(data1)
data2 = pd.read_csv('~/Desktop/data/data02_15_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
c = ks_stat_anom(data2)
cc = ks_stat_anomanom(data2)
data3 = pd.read_csv('~/Desktop/data/data02_16_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
d = ks_stat_anom(data3)
dd = ks_stat_anomanom(data3)
data4 = pd.read_csv('~/Desktop/data/data02_17_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
e = ks_stat_anom(data4)
ee = ks_stat_anomanom(data4)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (368). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (368). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


half way there
loop 0


/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (895). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (895). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


half way there
loop 0


/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (597). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (597). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


half way there
loop 0


/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (810). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (810). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


half way there
loop 0


/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (809). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (809). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


half way there
loop 0


In [172]:
# ece
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
a = ks_stat_anom(data0)
aa = ks_stat_anomanom(data0)
data1 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
b = ks_stat_anom(data1)
bb = ks_stat_anomanom(data1)
data2 = pd.read_csv('~/Desktop/data/data02_15_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
c = ks_stat_anom(data2)
cc = ks_stat_anomanom(data2)
data3 = pd.read_csv('~/Desktop/data/data02_16_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
d = ks_stat_anom(data3)
dd = ks_stat_anomanom(data3)
data4 = pd.read_csv('~/Desktop/data/data02_17_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
e = ks_stat_anom(data4)
ee = ks_stat_anomanom(data4)

half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0


In [173]:
stats.ks_2samp(a.rate,b.rate)

Ks_2sampResult(statistic=0.36500082467425365, pvalue=1.456614045939065e-10)

In [174]:
stats.ks_2samp(a.rate,c.rate)

Ks_2sampResult(statistic=0.36619718309859151, pvalue=1.0649800525139802e-11)

In [175]:
stats.ks_2samp(a.rate,d.rate)

Ks_2sampResult(statistic=0.5971428571428572, pvalue=2.2518990569389396e-32)

In [176]:
stats.ks_2samp(a.rate,e.rate)

Ks_2sampResult(statistic=0.57476635514018692, pvalue=1.6165079036311666e-25)

In [177]:
stats.ks_2samp(b.rate,c.rate)

Ks_2sampResult(statistic=0.1138388470357026, pvalue=0.077818225417967235)

In [178]:
stats.ks_2samp(b.rate,d.rate)

Ks_2sampResult(statistic=0.35142857142857142, pvalue=4.9128439361507686e-15)

In [179]:
stats.ks_2samp(b.rate,e.rate)

Ks_2sampResult(statistic=0.33214518582916763, pvalue=5.7966416144668597e-11)

In [180]:
stats.ks_2samp(c.rate,d.rate)

Ks_2sampResult(statistic=0.29716297786720325, pvalue=1.0640237211453144e-12)

In [181]:
stats.ks_2samp(c.rate,e.rate)

Ks_2sampResult(statistic=0.28458602079768336, pvalue=3.2566997077238797e-09)

In [182]:
stats.ks_2samp(d.rate,e.rate)

Ks_2sampResult(statistic=0.16739652870493987, pvalue=0.0009900648934823978)

In [183]:
stats.ks_2samp(aa.rate,bb.rate)

Ks_2sampResult(statistic=0.4845261121856867, pvalue=1.3505580704778089e-07)

In [184]:
stats.ks_2samp(aa.rate,cc.rate)

Ks_2sampResult(statistic=0.24134199134199136, pvalue=0.037126924908056962)

In [185]:
stats.ks_2samp(aa.rate,dd.rate)

Ks_2sampResult(statistic=0.49161831076724694, pvalue=8.299685369768486e-08)

In [186]:
stats.ks_2samp(aa.rate,ee.rate)

Ks_2sampResult(statistic=0.46590909090909094, pvalue=4.5968420637289112e-06)

In [187]:
stats.ks_2samp(bb.rate,cc.rate)

Ks_2sampResult(statistic=0.46504559270516715, pvalue=2.451089218212643e-13)

In [188]:
stats.ks_2samp(bb.rate,dd.rate)

Ks_2sampResult(statistic=0.17730496453900713, pvalue=0.020627616737057861)

In [193]:
stats.ks_2samp(dd.rate,dd.rate)

Ks_2sampResult(statistic=0.0, pvalue=1.0)

In [189]:
stats.ks_2samp(bb.rate,ee.rate)

Ks_2sampResult(statistic=0.21267730496453907, pvalue=0.016558836727314449)

In [190]:
stats.ks_2samp(cc.rate,dd.rate)

Ks_2sampResult(statistic=0.47298210064167517, pvalue=8.8083632376509241e-14)

In [191]:
stats.ks_2samp(cc.rate,ee.rate)

Ks_2sampResult(statistic=0.4464285714285714, pvalue=3.1520083054245033e-09)

In [192]:
stats.ks_2samp(dd.rate,ee.rate)

Ks_2sampResult(statistic=0.25221631205673756, pvalue=0.002360139889140027)

In [197]:
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]

In [199]:
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(data0)
y_pred = clf.predict(data0)
data0['ISO'] = y_pred

In [200]:
data0.head()

,bytes,pkts,dur,rate,ISO
0,14048,232,0.544,206588.24,1
1,7548,104,0.337,179181.01,1
2,17140,312,1.364,100527.86,1
3,3004,20,2.306,10421.51,1
4,3992,20,0.279,114465.95,1


In [203]:
def ks_stat_anom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
    clf.fit(data0)
    y_pred = clf.predict(data0)
    data0['ISO'] = y_pred
    normal = data0[data0.ISO == 1][["bytes","pkts","dur","rate"]]
    return normal

def ks_stat_anomanom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
    clf.fit(data0)
    y_pred = clf.predict(data0)
    data0['ISO'] = y_pred
    anomalies = data0[data0.ISO == -1]
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]
    index_normal = [] 
    for i in kse_result:
        if i[1] > 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom
    

In [217]:
# ece
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
a = ks_stat_anom(data0)
aa = ks_stat_anomanom(data0)
data1 = pd.read_csv('~/Desktop/data/data02_14_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
b = ks_stat_anom(data1)
bb = ks_stat_anomanom(data1)
data2 = pd.read_csv('~/Desktop/data/data02_15_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
c = ks_stat_anom(data2)
cc = ks_stat_anomanom(data2)
data3 = pd.read_csv('~/Desktop/data/data02_16_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
d = ks_stat_anom(data3)
dd = ks_stat_anomanom(data3)
data4 = pd.read_csv('~/Desktop/data/data02_17_morning_Ece', sep = ',')[["bytes","pkts","dur","rate"]]
e = ks_stat_anom(data4)
ee = ks_stat_anomanom(data4)

half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0


In [230]:
len(aa)

45

In [224]:
stats.ks_2samp(a.rate,aa.rate)

Ks_2sampResult(statistic=0.4699713370426572, pvalue=2.5829424516789237e-09)

In [225]:
stats.ks_2samp(b.rate,bb.rate)

Ks_2sampResult(statistic=0.59172571476189362, pvalue=2.3043890144166017e-35)

In [226]:
stats.ks_2samp(c.rate,cc.rate)

Ks_2sampResult(statistic=0.68888881393434886, pvalue=3.0691337090069178e-35)

In [227]:
stats.ks_2samp(d.rate,dd.rate)

Ks_2sampResult(statistic=0.56958393961978449, pvalue=6.2911590193170855e-33)

In [228]:
stats.ks_2samp(e.rate,ee.rate)

Ks_2sampResult(statistic=0.61984489444205082, pvalue=6.6337648142327447e-23)

In [223]:
# pages
data0 = pd.read_csv('~/Desktop/data/data02_13_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
a = ks_stat_anom(data0)
aa = ks_stat_anomanom(data0)
data1 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
b = ks_stat_anom(data1)
bb = ks_stat_anomanom(data1)
data2 = pd.read_csv('~/Desktop/data/data02_15_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
c = ks_stat_anom(data2)
cc = ks_stat_anomanom(data2)
data3 = pd.read_csv('~/Desktop/data/data02_16_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
d = ks_stat_anom(data3)
dd = ks_stat_anomanom(data3)
data4 = pd.read_csv('~/Desktop/data/data02_17_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
e = ks_stat_anom(data4)
ee = ks_stat_anomanom(data4)

half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0
half way there
loop 0


In [216]:
def ks_stat_anom(data0):
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    y_pred = clf.fit_predict(data0)
    data0['LOF'] = y_pred
    normal = data0[data0.LOF == 1][["bytes","pkts","dur","rate"]]
    return normal

def ks_stat_anomanom(data0):
    #data0 = pd.read_csv('~/Desktop/data/data02_14_morning_Pages', sep = ',')[["bytes","pkts","dur","rate"]]
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    y_pred = clf.fit_predict(data0)
    data0['LOF'] = y_pred
    anomalies = data0[data0.LOF == -1]
    anomalies = anomalies[["bytes","pkts","dur","rate"]]
    kse_result = KSE_Test(anomalies, int(0.5 * len(anomalies)))
    data0 = data0[["bytes","pkts","dur","rate"]]
    index_normal = [] 
    for i in kse_result:
        if i[1] > 0.5:
            index_normal.append(i[0])
    # final result is a data set of the anomalies of the anomalies
    anom = anomalies.iloc[index_normal]
    return anom
    